## 환경 설정

In [ ]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch>=1.8.1

!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 37.3 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292695 sha256=f7b50003d963e3537112434ea88538f77218b67e467cbbb184c9a1d1cbc532a0
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.4 MB/s eta 0:00:00
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-9f4seaw9/kobert-tokenizer_74f2f3b5f4ff407bab91355aa60a7983
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-9f4seaw9/kobert-tokenizer

In [ ]:
!pip install mecab-python3
!pip install konlpy
!pip install tensorflow-text
!pip install soynlp
!pip install -v python-mecab-ko
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 107.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 88.1 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled k

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import json
import os
import re
from konlpy.tag import Kkma
from soynlp.normalizer import *
from soynlp.tokenizer import LTokenizer
from konlpy.tag import Komoran
from transformers import BertTokenizer
import mecab
import sentencepiece as spm

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm.notebook import tqdm

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [ ]:
# Torch GPU 설정
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_type)

## 데이터

In [ ]:
train = pd.read_excel('/content/drive/MyDrive/WCRC/018.감성대화/Training_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Training/감성대화말뭉치(최종데이터)_Training.xlsx')
val = pd.read_excel('/content/drive/MyDrive/WCRC/018.감성대화/Validation_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Validation/감성대화말뭉치(최종데이터)_Validation.xlsx')

## 전처리

In [ ]:
train_df1 = train[['Unnamed: 0', '상황키워드', '감정_대분류', '감정_소분류', '사람문장1', '시스템문장1']]
train_df2 = train[['Unnamed: 0', '상황키워드', '감정_대분류', '감정_소분류', '사람문장2', '시스템문장2']]
train_df3 = train[['Unnamed: 0', '상황키워드', '감정_대분류', '감정_소분류', '사람문장3', '시스템문장3']]

# 열 이름 변경
train_df1 = train_df1.rename(columns={'시스템문장1': '시스템문장', '사람문장1': '사람문장'})
train_df2 = train_df2.rename(columns={'시스템문장2': '시스템문장', '사람문장2': '사람문장'})
train_df3 = train_df3.rename(columns={'시스템문장3': '시스템문장', '사람문장3': '사람문장'})

train_df = pd.concat([train_df1, train_df2, train_df3], ignore_index=True)

# NaN 값을 가진 행 삭제
train_df = train_df.dropna(subset=['사람문장', '시스템문장'])

# Unnamed: 0 기준으로 정렬
train_df = train_df.sort_values(by='Unnamed: 0')

# 인덱스 재설정
train_df = train_df.reset_index(drop=True)

In [ ]:
val_df1 = val[['Unnamed: 0', '상황키워드', '감정_대분류', '감정_소분류', '사람문장1', '시스템문장1']]
val_df2 = val[['Unnamed: 0', '상황키워드', '감정_대분류', '감정_소분류', '사람문장2', '시스템문장2']]
val_df3 = val[['Unnamed: 0', '상황키워드', '감정_대분류', '감정_소분류', '사람문장3', '시스템문장3']]

val_df1 = val_df1.rename(columns={'시스템문장1': '시스템문장', '사람문장1': '사람문장'})
val_df2 = val_df2.rename(columns={'시스템문장2': '시스템문장', '사람문장2': '사람문장'})
val_df3 = val_df3.rename(columns={'시스템문장3': '시스템문장', '사람문장3': '사람문장'})

val_df = pd.concat([val_df1, val_df2, val_df3], ignore_index=True)
val_df = val_df.dropna(subset=['사람문장', '시스템문장'])
val_df = val_df.sort_values(by='Unnamed: 0')
val_df = val_df.reset_index(drop=True)

In [ ]:
train_df['상황키워드'].value_counts()

대인관계            31376
진로,취업,직장        14606
재정              14584
건강,죽음           12340
가족관계            11768
연애,결혼,출산        11282
대인관계(부부, 자녀)     9641
학업 및 진로          9020
학교폭력/따돌림         8464
건강               8452
재정,은퇴,노후준비       7626
직장, 업무 스트레스      6795
Name: 상황키워드, dtype: int64

In [ ]:
train_df.loc[(train_df['상황키워드'] == "대인관계"), '상황키워드'] = 0
train_df.loc[(train_df['상황키워드'] == "진로,취업,직장"), '상황키워드'] = 1
train_df.loc[(train_df['상황키워드'] == "재정"), '상황키워드'] = 2
train_df.loc[(train_df['상황키워드'] == "건강,죽음"), '상황키워드'] = 3
train_df.loc[(train_df['상황키워드'] == "가족관계"), '상황키워드'] = 4
train_df.loc[(train_df['상황키워드'] == "연애,결혼,출산"), '상황키워드'] = 5
train_df.loc[(train_df['상황키워드'] == "대인관계(부부, 자녀)"), '상황키워드'] = 6
train_df.loc[(train_df['상황키워드'] == "학업 및 진로"), '상황키워드'] = 7
train_df.loc[(train_df['상황키워드'] == "학교폭력/따돌림"), '상황키워드'] = 8
train_df.loc[(train_df['상황키워드'] == "건강"), '상황키워드'] = 9
train_df.loc[(train_df['상황키워드'] == "재정,은퇴,노후준비"), '상황키워드'] = 10
train_df.loc[(train_df['상황키워드'] == "직장, 업무 스트레스"), '상황키워드'] = 11

In [ ]:
val_df.loc[(val_df['상황키워드'] == "대인관계"), '상황키워드'] = 0
val_df.loc[(val_df['상황키워드'] == "진로,취업,직장"), '상황키워드'] = 1
val_df.loc[(val_df['상황키워드'] == "재정"), '상황키워드'] = 2
val_df.loc[(val_df['상황키워드'] == "건강,죽음"), '상황키워드'] = 3
val_df.loc[(val_df['상황키워드'] == "가족관계"), '상황키워드'] = 4
val_df.loc[(val_df['상황키워드'] == "연애,결혼,출산"), '상황키워드'] = 5
val_df.loc[(val_df['상황키워드'] == "대인관계(부부, 자녀)"), '상황키워드'] = 6
val_df.loc[(val_df['상황키워드'] == "학업 및 진로"), '상황키워드'] = 7
val_df.loc[(val_df['상황키워드'] == "학교폭력/따돌림"), '상황키워드'] = 8
val_df.loc[(val_df['상황키워드'] == "건강"), '상황키워드'] = 9
val_df.loc[(val_df['상황키워드'] == "재정,은퇴,노후준비"), '상황키워드'] = 10
val_df.loc[(val_df['상황키워드'] == "직장, 업무 스트레스"), '상황키워드'] = 11

In [ ]:
train_df['감정_대분류'].value_counts()

불안    26301
분노    25999
상처    25957
슬픔    25814
당황    24936
기쁨    16947
Name: 감정_대분류, dtype: int64

In [ ]:
train_df.loc[(train_df['감정_대분류'] == "불안"), '감정_대분류'] = 0
train_df.loc[(train_df['감정_대분류'] == "분노"), '감정_대분류'] = 1
train_df.loc[(train_df['감정_대분류'] == "상처"), '감정_대분류'] = 2
train_df.loc[(train_df['감정_대분류'] == "슬픔"), '감정_대분류'] = 3
train_df.loc[(train_df['감정_대분류'] == "당황"), '감정_대분류'] = 4
train_df.loc[(train_df['감정_대분류'] == "기쁨"), '감정_대분류'] = 5

In [ ]:
val_df.loc[(val_df['감정_대분류'] == "불안"), '감정_대분류'] = 0
val_df.loc[(val_df['감정_대분류'] == "분노"), '감정_대분류'] = 1
val_df.loc[(val_df['감정_대분류'] == "상처"), '감정_대분류'] = 2
val_df.loc[(val_df['감정_대분류'] == "슬픔"), '감정_대분류'] = 3
val_df.loc[(val_df['감정_대분류'] == "당황"), '감정_대분류'] = 4
val_df.loc[(val_df['감정_대분류'] == "기쁨"), '감정_대분류'] = 5

In [ ]:
from konlpy.tag import Okt

def clean_korean_documents(documents):
    cleaned_documents = []

    for document in documents:
        okt = Okt()
        clean_words = []

        for word in okt.pos(document, stem=True):  # 어간 추출
            if word[1] in ['Noun', 'Verb', 'Adjective']:  # 명사, 동사, 형용사
                clean_words.append(word[0])

        cleaned_document = ' '.join(clean_words)
        cleaned_documents.append(cleaned_document)

    '''
    # 불용어 제거
    stopwords = ['을', '를', '이', '가', '은', '는', '의']  # 예제로 일부 불용어 추가
    cleaned_documents = [' '.join([word for word in document.split() if word not in stopwords]) for document in cleaned_documents]
    '''

    return cleaned_documents

# '사람문장' 열에 대해 텍스트 정제 적용
train_df['사람문장_텍스트'] = clean_korean_documents(train_df['사람문장'])
val_df['사람문장_텍스트'] = clean_korean_documents(val_df['사람문장'])

In [ ]:
from konlpy.tag import Okt

# 형태소 분석기 초기화
okt = Okt()

# '사람문장_텍스트' 열에서 명사 추출하여 새로운 열 생성
train_df['사람문장_명사'] = train_df['사람문장'].apply(lambda x: ' '.join([word for word in okt.nouns(x)]))
val_df['사람문장_명사'] = val_df['사람문장'].apply(lambda x: ' '.join([word for word in okt.nouns(x)]))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# '사람문장' 열에서 TF-IDF 벡터화
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(train_df['사람문장'])

# 각 단어의 중요도를 나타내는 특성 이름 가져오기
feature_names = vectorizer.get_feature_names_out()

# 중요한 단어 추출 (상위 5개)
top_5_important_words = []
for i, row in train_df.iterrows():
    tfidf_scores = zip(feature_names, tfidf_matrix[i].toarray()[0])
    sorted_tfidf_scores = sorted(tfidf_scores, key=lambda x: x[1], reverse=True)
    top_5_important_words.append([word for word, score in sorted_tfidf_scores[:5]])

# 결과를 데이터프레임에 추가
train_df['상위5중요단어'] = top_5_important_words

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# '사람문장' 열에서 TF-IDF 벡터화
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(val_df['사람문장'])

# 각 단어의 중요도를 나타내는 특성 이름 가져오기
feature_names = vectorizer.get_feature_names_out()

# 중요한 단어 추출 (상위 5개)
top_5_important_words = []
for i, row in val_df.iterrows():
    tfidf_scores = zip(feature_names, tfidf_matrix[i].toarray()[0])
    sorted_tfidf_scores = sorted(tfidf_scores, key=lambda x: x[1], reverse=True)
    top_5_important_words.append([word for word, score in sorted_tfidf_scores[:5]])

# 결과를 데이터프레임에 추가
val_df['상위5중요단어'] = top_5_important_words

In [ ]:
# 각 열의 값을 띄어쓰기로 연결하여 새로운 열 생성
train_df['상위5중요단어'] = train_df['상위5중요단어'].apply(lambda x: ' '.join(x))
val_df['상위5중요단어'] = val_df['상위5중요단어'].apply(lambda x: ' '.join(x))

In [ ]:
from konlpy.tag import Okt

# 형태소 분석기 초기화
okt = Okt()

# '상위5중요단어' 열의 값을 가져와서 명사만 추출하여 '상위5중요단어_명사' 열에 저장
train_df['상위5중요단어_명사'] = train_df['상위5중요단어'].apply(lambda x: ' '.join([word for word in okt.nouns(x)]))
val_df['상위5중요단어_명사'] = val_df['상위5중요단어'].apply(lambda x: ' '.join([word for word in okt.nouns(x)]))

In [ ]:
train_df.to_csv('/content/drive/MyDrive/WCRC/final_train_df.csv', index=False)
val_df.to_csv('/content/drive/MyDrive/WCRC/final_val_df.csv', index=False)

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/WCRC/final_train_df.csv')
val_df = pd.read_csv('/content/drive/MyDrive/WCRC/final_val_df.csv')